In [51]:
import openml
import pandas as pd
import numpy as np

In [52]:
openml_list = openml.datasets.list_datasets(output_format="dict")
datalist = pd.DataFrame.from_dict(openml_list, orient="index")
datalist["name"] = datalist.name.apply(lambda x: x.lower())

/tmp/ipykernel_70149/1902761381.py:1: FutureWarning: Support for `output_format` of 'dict' will be removed in 0.15 and pandas dataframes will be returned instead. To ensure your code will continue to work, use `output_format`='dataframe'.
  openml_list = openml.datasets.list_datasets(output_format="dict")


In [53]:
names = [
    "numerai28.6", #"numerai28_6",
    "sf-police-incidents",
    "airlines",
    "click_prediction_small",
    "albert",
    "kddcup09_appetency",
    "higgs",
    "christine",
    "guillermo",
    "amazon_employee_access",
]

In [54]:
info = datalist[datalist.name.apply(lambda x: x in names)]#.fillna(-1)
info["name"] = info["name"].apply(lambda x: x.lower())
replace_id = info[info["name"] == "numerai28.6"].index
info.loc[replace_id, "name"] = "numerai28_6"

def select_latest(df):
    latest = df.version.max()
    return df[df.version == latest]

info = info.groupby("name").apply(select_latest)

info = info.set_index("name")

/tmp/ipykernel_70149/1091183251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info["name"] = info["name"].apply(lambda x: x.lower())
/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [55]:
info

,did,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
name,,,,,,,,,,,,,,,
airlines,45072,4,31892,active,arff,NaN,NaN,NaN,0.0,8.0,539383.0,0.0,0.0,4.0,4.0
albert,45035,2,26324,active,arff,29126.0,NaN,29126.0,2.0,32.0,58252.0,0.0,0.0,21.0,11.0
amazon_employee_access,43900,2,30861,active,ARFF,30872.0,NaN,1897.0,2.0,10.0,32769.0,0.0,0.0,0.0,10.0
christine,41142,1,1478,active,ARFF,2709.0,2.0,2709.0,2.0,1637.0,5418.0,0.0,0.0,1599.0,38.0
click_prediction_small,43901,11,30861,active,ARFF,33201.0,NaN,6725.0,2.0,9.0,39926.0,0.0,0.0,4.0,5.0
guillermo,41159,1,1478,active,ARFF,11997.0,2.0,8003.0,2.0,4297.0,20000.0,0.0,0.0,4296.0,1.0
higgs,45570,8,86,active,arff,5829123.0,NaN,5170877.0,2.0,29.0,11000000.0,0.0,0.0,28.0,1.0
kddcup09_appetency,1111,1,2,active,ARFF,49110.0,15415.0,890.0,2.0,231.0,50000.0,50000.0,8024152.0,192.0,39.0
numerai28_6,23517,2,2,active,ARFF,48658.0,2.0,47662.0,2.0,22.0,96320.0,0.0,0.0,21.0,1.0


In [56]:
def calc_majority_class_size(df, name):
    return df["y"].value_counts().max()

def calc_minority_class_size(df, name):
    return df["y"].value_counts().min()

def calc_number_of_classes(df, name):
    return len(df["y"].value_counts())
    # return info.loc[name]["NumberOfClasses"]

def calc_number_of_features(df, name):
    return info.loc[name]["NumberOfFeatures"]

def calc_number_of_instances(df, name):
    return len(df)

def calc_number_of_instances_with_missing_values(df, name):
    return df.isna().sum(axis=1).astype(bool).astype(int).sum()

def calc_number_of_missing_values(df, name):
    return df.isna().to_numpy().sum()

def calc_number_of_numeric_features(df, name):
    return info.loc[name]["NumberOfNumericFeatures"]

def calc_number_of_symbolic_features(df, name):
    return info.loc[name]["NumberOfSymbolicFeatures"]

def extract(df, name):
    return {
        "MajorityClassSize": calc_majority_class_size(df, name),
        "MinorityClassSize": calc_minority_class_size(df, name),
        "NumberOfClasses": calc_number_of_classes(df, name),
        "NumberOfFeatures": calc_number_of_features(df, name),
        "NumberOfInstances": calc_number_of_instances(df, name),
        "NumberOfInstancesWithMissingValues": calc_number_of_instances_with_missing_values(df, name),
        "NumberOfMissingValues": calc_number_of_missing_values(df, name),
        "NumberOfNumericFeatures": calc_number_of_numeric_features(df, name),
        "NumberOfSymbolicFeatures": calc_number_of_symbolic_features(df, name),

    }

In [57]:
import os
from tqdm.notebook import tqdm


folds_folder = "/home/cherniak/itmo_job/datasets_folds"

meta_features = {}
for dataset_name in tqdm(names):
    if dataset_name == "numerai28.6":
        dataset_name = "numerai28_6"
    for dataset_fold in range(10):
        x_fname = f"train_{dataset_name}_fold{dataset_fold}.npy"
        y_fname = f"trainy_{dataset_name}_fold{dataset_fold}.npy"
        x = np.load(os.path.join(folds_folder, x_fname))
        y = np.load(os.path.join(folds_folder, y_fname))
        df = pd.DataFrame(data=x)
        df["y"] = y
        ft = extract(df, dataset_name)
        dataset_id = f"{dataset_name}_{dataset_fold}"
        meta_features[dataset_id] = ft

  0%|          | 0/10 [00:00<?, ?it/s]

In [58]:
df = pd.DataFrame.from_dict(meta_features).T
print(df.isna().sum())
df.describe()

MajorityClassSize                     0
MinorityClassSize                     0
NumberOfClasses                       0
NumberOfFeatures                      0
NumberOfInstances                     0
NumberOfInstancesWithMissingValues    0
NumberOfMissingValues                 0
NumberOfNumericFeatures               0
NumberOfSymbolicFeatures              0
dtype: int64


,MajorityClassSize,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
count,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000,1.000000e+02,100.000000,100.000000
mean,3.863570e+05,70502.950000,1344.190000,628.400000,4.931652e+05,42764.400000,9.682345e+05,616.800000,11.600000
std,6.361680e+05,97609.228815,3826.493687,1320.053121,7.229494e+05,114659.268134,2.220957e+06,1319.877772,13.927663
min,2.700000e+01,1.000000,2.000000,8.000000,4.876000e+03,0.000000,0.000000e+00,0.000000,1.000000
25%,1.079700e+04,801.000000,2.000000,9.000000,2.949200e+04,0.000000,0.000000e+00,4.000000,1.000000
50%,4.399600e+04,4820.000000,2.000000,25.500000,8.746650e+04,0.000000,0.000000e+00,21.000000,5.500000
75%,2.692070e+05,191358.000000,2.000000,231.000000,4.854450e+05,1.000000,9.000000e+00,192.000000,11.000000
max,1.751134e+06,242388.000000,12790.000000,4297.000000,1.993521e+06,382649.000000,7.222197e+06,4296.000000,39.000000


In [59]:
to_drop = ["NumberOfInstancesWithMissingValues", "NumberOfMissingValues"]
df = df.drop(to_drop, axis=1)

In [60]:
df

,MajorityClassSize,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfSymbolicFeatures
numerai28_6_0,43793.0,42895.0,2.0,22.0,86688.0,21.0,1.0
numerai28_6_1,43793.0,42895.0,2.0,22.0,86688.0,21.0,1.0
numerai28_6_2,43792.0,42896.0,2.0,22.0,86688.0,21.0,1.0
numerai28_6_3,43792.0,42896.0,2.0,22.0,86688.0,21.0,1.0
numerai28_6_4,43792.0,42896.0,2.0,22.0,86688.0,21.0,1.0
...,...,...,...,...,...,...,...
amazon_employee_access_5,27785.0,1707.0,2.0,10.0,29492.0,0.0,10.0
amazon_employee_access_6,27785.0,1707.0,2.0,10.0,29492.0,0.0,10.0
amazon_employee_access_7,27785.0,1707.0,2.0,10.0,29492.0,0.0,10.0
amazon_employee_access_8,27785.0,1707.0,2.0,10.0,29492.0,0.0,10.0


In [61]:
df.to_csv("/home/cherniak/itmo_job/surrogate/scripts/evaluate_on_test_datasets/openml_meta_feats.csv", index=True)